## ExMAS
> Equilibrium matching


In [1]:
import os
import math
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching
from ExMAS.extras import games, pricings, prunings, pipeline

In [4]:
params = ExMAS.utils.get_config('ExMAS/spinoffs/game/pipe.json') # load the default 
params.t0 = pd.to_datetime(params.t0)
params.matching_obj = 'u_pax'
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph
params.nP = 50
params.simTime = 0.1
params.shared_discount = 0.3
inData = ExMAS.utils.generate_demand(inData, params)  # generate requests

In [6]:
from ExMAS.main import init_log
params.logger_level = 'WARNING'
inData.logger = init_log(params)

In [7]:
inData = ExMAS.main(inData, params, plot = False)
KPIs = inData.sblts.res.to_frame('u_pax')

12-02-21 10:02:57-WARNING-Prepare for game:  1/6
12-02-21 10:02:57-WARNING-Prepare for game:  2/6
12-02-21 10:02:58-WARNING-Prepare for game:  3/6
12-02-21 10:02:58-WARNING-Prepare for game:  4/6
12-02-21 10:03:08-WARNING-Prepare for game:  5/6
12-02-21 10:03:09-WARNING-Prepare for game:  6/6


In [8]:

params.time_cost = params.VoT # travellers' cost per travel time
params.wait_cost = params.time_cost*1.5 # and waiting
params.sharing_penalty_fixed = 0 # fixed penalty (EUR) per 
params.sharing_penalty_multiplier = 0.2 # fixed penalty (EUR) per 

params.veh_cost = 2.3*params.VoT/params.avg_speed # operating costs per kilometer
params.fixed_ride_cost = 0.5 # ride fixed costs (per vehicle)

In [9]:
inData = games.prepare_PoA(inData)

12-02-21 10:03:09-WARNING-Prepare for game:  1/6
12-02-21 10:03:09-WARNING-Prepare for game:  2/6
12-02-21 10:03:10-WARNING-Prepare for game:  3/6
12-02-21 10:03:10-WARNING-Prepare for game:  4/6
12-02-21 10:03:19-WARNING-Prepare for game:  5/6
12-02-21 10:03:20-WARNING-Prepare for game:  6/6


### pricings with various strategies

In [28]:
inData = ExMAS.extras.pricings.update_costs(inData, params)
inData = pricings.uniform_split(inData) 
inData = pricings.externality_split(inData) 
inData = pricings.residual_split(inData) 
inData = pricings.subgroup_split(inData) 

In [29]:
PRUNINGS = dict()  # algorithms to apply and their names
PRUNINGS['TSE'] = prunings.algo_TSE
for PRUNING, pruning in PRUNINGS.items():
    inData = pruning(inData, price_column=PRICING)  # apply pruning strategies for a given pricing strategy

In [30]:
ret = dict()
params.logger_level = 'INFO'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params,
                                 EXPERIMENT_NAME='jupyter',
                                 MATCHING_OBJS=['total_group_cost'],  # this can be more
                                 PRUNINGS=["TSE"],  # and this can be more
                                 PRICING=PRICING,  # this is taken from first level loop
                                 minmax=('min', 'max'), store_res = False)  # direction BPoA, WPoA
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['matched'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','matched', 'obj']].copy()
pd.DataFrame(ret).T

12-02-21 10:39:42-WARNING-Prunings:  ['TSE']
12-02-21 10:39:42-WARNING-Pruned nRides 464/464
12-02-21 10:39:42-WARNING-Mutually exclusives 0
12-02-21 10:39:42-WARNING-Experiment-jupyter_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['TSE']_minmax-min
12-02-21 10:39:42-INFO-DotMap(VehHourTrav=14205, VehHourTrav_ns=15897, PassHourTrav=18721, PassHourTrav_ns=15897, PassUtility=241.429275, PassUtility_ns=246.675, mean_lambda=0.36098208360982087, revenue_s=16691.85, revenue_ns=23845.5, Fare_Discount=-0.30000000000000004, nR=50, SINGLE=39, PAIRS=4, TRIPLES=1, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.21999999999999997, fleet_size_nonshared=35.0, fleet_size_shared=30.0, lambda_shared=0.7656744261497282)
12-02-21 10:39:42-WARNING-Experiment-jupyter_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['TSE']_minmax-max
12-02-21 10:39:43-INFO-DotMap(VehHourTrav=14205, VehHourTrav_ns=15897, PassHourTrav=18721, PassHourTrav_ns=15897, PassUtility=241.429275, PassUtility_ns=246.6

,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,matched,obj
UNIFORM,14205,18721,0.22,136.833,63.2063,44,200.04
EXTERNALITY,14205,18721,0.22,136.833,63.2063,44,200.04
RESIDUAL,14205,18721,0.22,136.833,63.2063,44,-8.57077
SUBGROUP,14205,18721,0.22,136.833,63.2063,44,200.04


In [31]:
PRICING = 'UNIFORM'

ride  traveller  shared  degree  treq ride_time  dist  ttrav  \
0   0      0          0   False       1     0       144  1159    144   
1   1      1          1   False       1     1       607  4856    607   
2   2      2          2   False       1     1       393  3151    393   
3   3      3          3   False       1     6       305  2447    305   
4   4      4          4   False       1     9       330  2646    330   
...      ...        ...     ...     ...   ...       ...   ...    ...   
463 1    463          1    True       5     1       845  4856    607   
    11   463         11    True       5    55       845  2261    282   
    40   463         40    True       5   254       845  1811    226   
    39   463         39    True       5   253       845  3124    390   
    38   463         38    True       5   251       845  3917    489   

        ttrav_sh  delay distance cost_veh  cost_user  total_group_cost  \
0   0        144    0.0     1152   1.6592    0.50400           2.16320   
1   1        607    0.0     4856  5.38635    2.12450           7.51085   
2   2        393    0.0     3144  3.66365    1.37550           5.03915   
3   3        305    0.0     2440  2.95525    1.06750           4.02275   
4   4        330    0.0     2640   3.1565    1.15500           4.31150   
...          ...    ...      ...      ...        ...               ...   
463 1        687  -26.0     6760  7.30225    3.02190          20.86720   
    11       484   41.0     6760  7.30225    2.24805          20.86720   
    40       438 -112.0     6760  7.30225    2.42760          20.86720   
    39       657  -90.0     6760  7.30225    3.23190          20.86720   
    38       595  -26.0     6760  7.30225    2.63550          20.86720   

        cost_single  total_singles  residual_user  UNIFORM  desired_UNIFORM  \
0   0       2.16320        2.16320         0.0000  2.16320         2.163200   
1   1       7.51085        7.51085         0.0000  7.51085         3.937375   
2   2       5.03915        5.03915         0.0000  5.03915         4.292150   
3   3       4.02275        4.02275         0.0000  4.02275         4.022750   
4   4       4.31150        4.31150         0.0000  4.31150         4.152150   
...             ...            ...            ...      ...              ...   
463 1       7.51085       25.53070        -4.6635  4.17344         3.937375   
    11      3.75710       25.53070        -4.6635  4.17344         3.107575   
    40      3.11030       25.53070        -4.6635  4.17344         2.776300   
    39      5.00450       25.53070        -4.6635  4.17344         3.132567   
    38      6.14795       25.53070        -4.6635  4.17344         3.573470   

        EXTERNALITY  desired_EXTERNALITY  RESIDUAL  desired_RESIDUAL  \
0   0       2.16320              2.16320  2.163200          2.163200   
1   1       7.51085              0.00000  7.510850          5.759677   
2   2       5.03915              4.93430  5.039150          4.988913   
3   3       4.02275              4.02275  4.022750          4.022750   
4   4       4.31150              4.07365  4.311500          4.191449   
...             ...                  ...       ...               ...   
463 1       0.00000              0.00000  6.138900          5.759677   
    11      3.13005              0.00000  3.070819          2.901031   
    40      2.48605              0.00000  2.542165          2.346855   
    39      2.83325              0.00000  4.090366          3.776110   
    38      5.11770              0.00000  5.024950          4.678361   

        price_subgroup  SUBGROUP  desired_SUBGROUP  pruned_EXMAS  \
0   0         2.163200  2.163200          2.163200          True   
1   1         7.510850  7.510850          5.759677          True   
2   2         5.039150  5.039150          4.988913          True   
3   3         4.022750  4.022750          4.022750          True   
4   4         4.152150  4.311500          4.191449          True   
...                ...       ...

# Differences between algorithms and pricings

## TNE

In [12]:
ret = dict()
params.logger_level = 'INFO'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params, prunings.algo_TNE, PRICING = PRICING, ALGO = 'TNE', minmax = ['min'], store_res = False)
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['pruned'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','pruned', 'obj']]
pd.DataFrame(ret).T

15-12-20 16:21:10-WARNING-Pruned nRides 51/222
15-12-20 16:21:10-WARNING-UNIFORM-TNE-UNIFORM-min
15-12-20 16:21:11-INFO-Matching 50 trips to 222 rides in order to min UNIFORM
15-12-20 16:21:11-INFO-Problem solution: Optimal. 
Total costs for single trips:            162 
reduced by matching to:                  159
15-12-20 16:21:11-INFO-DotMap(VehHourTrav=15108, VehHourTrav_ns=17058, PassHourTrav=17711, PassHourTrav_ns=17058, PassUtility=253.51685, PassUtility_ns=264.633, mean_lambda=0.30019524894240157, revenue_s=17910.899999999998, revenue_ns=25587.0, Fare_Discount=-0.3000000000000001, nR=50, SINGLE=36, PAIRS=7, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.28, fleet_size_nonshared=45.0, fleet_size_shared=38.0, lambda_shared=0.30019524894240157)
15-12-20 16:21:11-WARNING-Pruned nRides 61/222
15-12-20 16:21:11-WARNING-EXTERNALITY-TNE-EXTERNALITY-min
15-12-20 16:21:12-INFO-Matching 50 trips to 222 rides in order to min EXTERNALITY
15-12-20 16:21:12-INFO-Problem solution

,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,pruned,obj
UNIFORM,15108,17711,0.28,90.7192,68.9161,43,159.635
EXTERNALITY,15108,17711,0.28,90.7192,68.9161,43,159.635
RESIDUAL,15108,17711,0.28,90.7192,68.9161,43,-2.6817
SUBGROUP,15815,19702,0.76,88.755,100.877,31,162.314


## Hermetic

In [13]:
ret = dict()
params.logger_level = 'WARNING'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params, prunings.algo_HERMETIC, PRICING = PRICING, ALGO = 'HERMETIC', minmax = ['min'], store_res = False)
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['pruned'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','pruned', 'obj']]
pd.DataFrame(ret).T

15-12-20 16:21:15-WARNING-Pruned nRides 51/222
15-12-20 16:21:15-WARNING-UNIFORM-HERMETIC-UNIFORM-min
15-12-20 16:21:16-WARNING-Pruned nRides 58/222
15-12-20 16:21:16-WARNING-EXTERNALITY-HERMETIC-EXTERNALITY-min
15-12-20 16:21:17-WARNING-Pruned nRides 58/222
15-12-20 16:21:17-WARNING-RESIDUAL-HERMETIC-RESIDUAL-min
15-12-20 16:21:20-WARNING-Pruned nRides 222/222
15-12-20 16:21:20-WARNING-SUBGROUP-HERMETIC-SUBGROUP-min


,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,pruned,obj
UNIFORM,15108,17711,0.28,90.7192,68.9161,43,159.635
EXTERNALITY,15108,17711,0.28,90.7192,68.9161,43,159.635
RESIDUAL,15108,17711,0.28,90.7192,68.9161,43,-2.6817
SUBGROUP,15815,19702,0.76,88.755,100.877,31,162.314


## RUE

In [14]:
ret = dict()
params.logger_level = 'INFO'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params, prunings.algo_RUE, PRICING = PRICING, ALGO = 'RUE', minmax = ['min'], store_res = False)
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['pruned'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','pruned', 'obj']]
pd.DataFrame(ret).T

15-12-20 16:21:21-INFO-Mergeable groups: 25-30
15-12-20 16:21:22-INFO-Mergeable groups: 5-88
15-12-20 16:21:22-INFO-Mergeable groups: 47-148
15-12-20 16:21:22-WARNING-Pruned nRides 222/222
15-12-20 16:21:22-WARNING-UNIFORM-RUE-UNIFORM-min
15-12-20 16:21:22-INFO-Matching 50 trips to 222 rides in order to min UNIFORM
15-12-20 16:21:22-INFO-Adding 3 mutually exlcusive constrains
15-12-20 16:21:22-INFO-Problem solution: Optimal. 
Total costs for single trips:            162 
reduced by matching to:                  159
15-12-20 16:21:22-INFO-DotMap(VehHourTrav=15108, VehHourTrav_ns=17058, PassHourTrav=17711, PassHourTrav_ns=17058, PassUtility=253.51685, PassUtility_ns=264.633, mean_lambda=0.30019524894240157, revenue_s=17910.899999999998, revenue_ns=25587.0, Fare_Discount=-0.3000000000000001, nR=50, SINGLE=36, PAIRS=7, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.28, fleet_size_nonshared=45.0, fleet_size_shared=38.0, lambda_shared=0.30019524894240157)
15-12-20 16:21:22-INFO

,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,pruned,obj
UNIFORM,15108,17711,0.28,90.7192,68.9161,43,159.635
EXTERNALITY,15108,17711,0.28,90.7192,68.9161,43,159.635
RESIDUAL,15108,17711,0.28,90.7192,68.9161,43,-2.6817
SUBGROUP,15815,19702,0.76,88.755,100.877,31,162.314


## RSIE

In [15]:
ret = dict()
params.logger_level = 'INFO'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params, prunings.algo_RSIE, PRICING = PRICING, ALGO = 'RSIE', minmax = ['min'], store_res = False)
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['pruned'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','pruned', 'obj']]
pd.DataFrame(ret).T

15-12-20 16:21:27-WARNING-Searching unstable pairs 0/222. 0 found so far
15-12-20 16:21:36-WARNING-Searching unstable pairs 20/222. 96 found so far
15-12-20 16:21:44-WARNING-Searching unstable pairs 40/222. 157 found so far
15-12-20 16:21:51-WARNING-Searching unstable pairs 60/222. 186 found so far
15-12-20 16:21:56-WARNING-Searching unstable pairs 80/222. 193 found so far
15-12-20 16:22:01-WARNING-Searching unstable pairs 100/222. 205 found so far
15-12-20 16:22:06-WARNING-Searching unstable pairs 120/222. 205 found so far
15-12-20 16:22:10-WARNING-Searching unstable pairs 140/222. 210 found so far
15-12-20 16:22:15-WARNING-Searching unstable pairs 160/222. 239 found so far
15-12-20 16:22:19-WARNING-Searching unstable pairs 180/222. 239 found so far
15-12-20 16:22:24-WARNING-Searching unstable pairs 200/222. 240 found so far
15-12-20 16:22:28-WARNING-Searching unstable pairs 220/222. 240 found so far
15-12-20 16:22:28-WARNING-Pruned nRides 222/222
15-12-20 16:22:28-WARNING-UNIFORM-RSI

,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,pruned,obj
UNIFORM,15108,17711,0.28,90.7192,68.9161,43,159.635
EXTERNALITY,15108,17711,0.28,90.7192,68.9161,43,159.635
RESIDUAL,15108,17711,0.28,90.7192,68.9161,43,-2.6817
SUBGROUP,15197,19623,0.8,85.5113,95.9004,30,162.314


## TSE

In [16]:
ret = dict()
params.logger_level = 'INFO'
inData.logger = init_log(params)
for PRICING in ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']:
    inData = pipeline.single_eval(inData, params, prunings.algo_TSE, PRICING = PRICING, ALGO = 'TSE', minmax = ['min'], store_res = False)
    inData.sblts.res['costs_veh'] = inData.sblts.rides[inData.sblts.rides.selected==1]['costs_veh'].sum()
    inData.sblts.res['obj'] = inData.sblts.rides[inData.sblts.rides.selected==1][PRICING].sum()
    inData.sblts.res['costs_user'] = inData.sblts.rides[inData.sblts.rides.selected==1][ 'costs_user'].sum()
    inData.sblts.res['pruned'] = inData.sblts.rides[inData.sblts.rides.selected==1].shape[0]
    ret[PRICING] = inData.sblts.res.loc[['VehHourTrav','PassHourTrav','shared_ratio','costs_veh', 'costs_user','pruned', 'obj']]
pd.DataFrame(ret).T

15-12-20 16:25:26-WARNING-Pruned nRides 222/222
15-12-20 16:25:26-INFO-DotMap(VehHourTrav=16748, VehHourTrav_ns=17058, PassHourTrav=19623, PassHourTrav_ns=17058, PassUtility=248.87515, PassUtility_ns=264.633, mean_lambda=0.30825496342737724, revenue_s=17910.899999999998, revenue_ns=25587.0, Fare_Discount=-0.3000000000000001, nR=50, SINGLE=48, PAIRS=1, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.040000000000000036, fleet_size_nonshared=45.0, fleet_size_shared=44.0, lambda_shared=0.953082919914954)
15-12-20 16:25:26-WARNING-Pruned nRides 222/222
15-12-20 16:25:26-INFO-DotMap(VehHourTrav=16748, VehHourTrav_ns=17058, PassHourTrav=19623, PassHourTrav_ns=17058, PassUtility=248.87515, PassUtility_ns=264.633, mean_lambda=0.30825496342737724, revenue_s=17910.899999999998, revenue_ns=25587.0, Fare_Discount=-0.3000000000000001, nR=50, SINGLE=48, PAIRS=1, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.040000000000000036, fleet_size_nonshared=45.0, fleet_size_shared=4

,VehHourTrav,PassHourTrav,shared_ratio,costs_veh,costs_user,pruned,obj
UNIFORM,16748,19623,0.04,100.772,61.3221,49,162.094
EXTERNALITY,16748,19623,0.04,100.772,61.3221,49,162.094
RESIDUAL,15108,19623,0.28,90.7192,68.9161,43,-2.6817
SUBGROUP,17058,19623,0,102.614,59.703,50,162.314


---
(c) Rafał Kucharski, Delft, 2020